In [2]:
options(warning = -1)
options("getSymbols.warning4.0"=FALSE)
library(quantmod)

Warning message:
"package 'quantmod' was built under R version 3.6.3"
Loading required package: xts

Warning message:
"package 'xts' was built under R version 3.6.3"
Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 3.6.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: TTR

Warning message:
"package 'TTR' was built under R version 3.6.3"
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Version 0.4-0 included new data defaults. See ?getSymbols.



In [26]:
#Empresa e benchmark
getSymbols("VALE3.SA", from = "2019-07-09", to = "2020-07-09")
getSymbols("^BVSP", from = "2019-07-09", to = "2020-07-09")
petr <- na.omit(VALE3.SA)
ibov <- na.omit(BVSP)

Warning message:
"VALE3.SA contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them."


[1] "VALE3.SA"

Warning message:
"^BVSP contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them."


[1] "^BVSP"

In [27]:
#Calculando o Beta
varibov <- na.omit(diff(ibov$BVSP.Close,1)/ibov$BVSP.Close[1:nrow(ibov)-1])
varpetr <- na.omit(diff(petr$VALE3.SA.Close,1)/petr$VALE3.SA.Close[1:nrow(petr)-1])
beta <- cov(varibov, varpetr)/(sd(varibov)^2)

In [6]:
#Taxa de juros
tx_juros <- 0.0225

In [3]:
#Fluxo de caixa livre e lucro líquido (Possíveis escolhas)
fluxo <- c(27510.05*1000000, 33952.24*1000000, 32509.78*1000000)
lucro <- c(17627.2*1000000, 25656.53*1000000, -6671.44*100000)
crescimento_fluxo <- (last(fluxo)/first(fluxo))^(1/length(fluxo))-1

In [4]:
#Dados da empresa
n_acoes <- 5284474782
vpa <- 32.72
fpa <- fluxo/n_acoes
lpa <- lucro/n_acoes
ativos <- 439706999000
divida <- 97573517000
pc_ativo <- ativos/(ativos+divida)
pc_divida <- divida/(ativos+divida)
juros <- 0.06
imposto <- 0.34

In [5]:
#Calculando o wacc
capm <- tx_juros+beta*(crescimento_fluxo-tx_juros)
wacc <- (capm*pc_ativo)+(juros*pc_divida*(1-imposto))

ERROR: Error in eval(expr, envir, enclos): objeto 'tx_juros' não encontrado


In [39]:
#Terminando o valuation
crescimento_f <- 0.03
A <- (1+crescimento_fluxo)/(1+wacc)
B <- (1+crescimento_f)/(1+wacc)
valor_3anos <- (last(fpa)*A*(1-A^(3))/(1-A))
valor_perpetuo <- last(fpa)*A^3*B*(1-B^3)/(1-B)
total_valor <-valor_3anos+valor_perpetuo
rownames(total_valor) <- c("Valor Intrínsico")
colnames(total_valor) <- c("VALE3")
total_valor

,VALE3
Valor Intrínsico,36.53321


In [36]:
#Margem de segurança em %
margem <- as.numeric(total_valor/last(petr$VALE3.SA.Close)-1)*100
margem

[1] 20.81857

In [20]:
#Fórmula de Benjamin Graham
graham <- (round(sqrt(last(lpa)*vpa*22.5), 2))
names(graham) <- "Peço Intrínsico por Graham"
if(is.nan(graham)){
    graham <- "A empresa não dá lucro ou tem vpa negativo"
}
graham

Warning message in sqrt(last(lpa) * vpa * 22.5):
"NaNs produzidos"


[1] "A empresa não dá lucro ou tem vpa negativo"